In [33]:
from xgboost import XGBRegressor
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.feature_selection import RFECV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_squared_log_error, make_scorer
from sklearn.preprocessing import RobustScaler, PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
import numpy as np
import matplotlib.pyplot as plt
import json
from joblib import dump
import shap

# Load data
df = pd.read_csv('/Users/skylerwilson/Desktop/PartsWise/co-pilot-v1/data/processed_data/parts_data_prepared.csv')

feature_cols = [col for col in df.columns if col not in {'part_number', 'description', 'supplier_name',
                                                         'sales_last_jan','sales_last_feb', 'sales_last_mar', 'sales_last_apr', 'sales_last_may',
                                                         'sales_last_jun', 'sales_last_jul', 'sales_last_aug', 'sales_last_sep',
                                                         'sales_last_oct', 'sales_last_nov', 'sales_last_dec', 'sales_jan',
                                                         'sales_feb', 'sales_mar', 'sales_apr', 'sales_may', 'sales_jun', 
                                                         'sales_jul', 'sales_aug', 'sales_sep', 'sales_oct', 'sales_nov', 
                                                         'sales_dec', 'sales_revenue', 'cogs', 'price', 'rolling_12m_sales'}]

mask = df['months_no_sale'] >=12                          
X = df[~mask][feature_cols]

y = df[~mask]['rolling_12m_sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

# Define preprocessing pipeline
numerical_features = X_train.select_dtypes(include=['int64', 'float64']).columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('scaler', RobustScaler()),
            ('power_trans', PowerTransformer(method='yeo-johnson'))]),
        numerical_features)
    ])

X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

# Define space for Hyperopt
space = {
    'objective': 'reg:pseudohubererror',
    'colsample_bytree': hp.uniform('colsample_bytree', 0.4, 1.0),
    'gamma': hp.uniform('gamma', 0.25, 1.0),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.05)),
    'max_depth': hp.quniform('max_depth', 5, 15, 1),
    'min_child_weight': hp.quniform('min_child_weight', 3, 15, 1),
    'n_estimators': hp.quniform('n_estimators', 350, 750, 10),
    'reg_alpha': hp.loguniform('reg_alpha', np.log(0.0001), np.log(1)),
    'reg_lambda': hp.loguniform('reg_lambda', np.log(1), np.log(3)),
    'subsample': hp.uniform('subsample', 0.5, 1.0),
    'max_delta_step': hp.quniform('max_delta_step', 5, 10, 1),
    'huber_slope': hp.uniform('huber_slope', 0.2, 0.3),
}

# Set up KFold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Set up the scoring function for cross-validation
scorer = make_scorer(mean_squared_log_error, greater_is_better=False)

def objective(params):
    params['n_estimators'] = int(params['n_estimators'])
    params['max_depth'] = int(params['max_depth'])
    params['min_child_weight'] = int(params['min_child_weight'])
    params['max_delta_step'] = int(params['max_delta_step']) 
    
    model = XGBRegressor(**params)
    # Store the scores for each fold using MSLE scorer
    scores = cross_val_score(model, X_train_transformed, y_train, scoring=scorer, cv=5)
    return {'loss': -scores.mean(), 'status': STATUS_OK}

# Run the optimization
trials = Trials()
best_hyperparams = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=500, trials=trials)

print("Best Hyperparameters:", best_hyperparams)

# Correctly casting the best hyperparameters to their correct types
best_hyperparams['n_estimators'] = int(best_hyperparams['n_estimators'])
best_hyperparams['max_depth'] = int(best_hyperparams['max_depth'])
best_hyperparams['max_delta_step'] = int(best_hyperparams['max_delta_step'])
best_hyperparams['min_child_weight'] = int(best_hyperparams['min_child_weight'])

# Train the model with the best hyperparameters
model = XGBRegressor(**best_hyperparams)

rfecv = RFECV(estimator=model, step=1, cv=KFold(10), scoring='neg_mean_absolute_error')

# Fit RFE
rfecv.fit(X_train_transformed, y_train)

selected_features_mask = rfecv.support_

# Get the ranking of the features
feature_ranking = rfecv.ranking_

# Get the selected features from your preprocessor
selected_features = [feature for feature, selected in zip(numerical_features, selected_features_mask) if selected]

# Print the optimal number of features
print(f"Optimal number of features: {rfecv.n_features_}")

# Save the selected features and the best hyperparameters to a JSON file
results_dict = {
    "selected_features": selected_features,
    "best_hyperparameters": best_hyperparams
}

with open('/Users/skylerwilson/Desktop/PartsWise/co-pilot-v1/Dashboard/Models/demand_predictor/general_model_details.json', 'w') as fp:
    json.dump(results_dict, fp, indent=4)

# Plot number of features VS. cross-validation scores
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (negative MAE)")
plt.plot(range(1, len(rfecv.cv_results_['mean_test_score']) + 1), rfecv.cv_results_['mean_test_score'])
plt.show()

# Transform the model to use the best features
X_train_transformed_rfe = rfecv.transform(X_train_transformed)
X_test_transformed_rfe = rfecv.transform(X_test_transformed)

model.fit(X_train_transformed_rfe, y_train)

# Save the preprocessor and the model for application
dump(preprocessor, '/Users/skylerwilson/Desktop/PartsWise/co-pilot-v1/dashboard/models/demand_predictor/preprocessor.joblib')
dump(model, '/Users/skylerwilson/Desktop/PartsWise/co-pilot-v1/dashboard/models/demand_predictor/xgb_regressor_with_selected_features.joblib')

# Get the feature importances
feature_importances = model.feature_importances_
importance_df = pd.DataFrame({'feature': selected_features, 'importance': feature_importances})

# Save the feature importances
importance_df.to_csv('/Users/skylerwilson/Desktop/PartsWise/co-pilot-v1/Dashboard/Models/demand_predictor/feature_importances.csv', index=False)

# Use SHAP for analysis
explainer = shap.Explainer(model, X_train_transformed_rfe)
shap_values = explainer(X_test_transformed_rfe)

# Save SHAP values
shap_df = pd.DataFrame(shap_values.values, columns=selected_features)
shap_df.to_csv('/Users/skylerwilson/Desktop/PartsWise/co-pilot-v1/Dashboard/Models/demand_predictor/shap_values.csv', index=False)

# Plot SHAP summary
shap.summary_plot(shap_values, X_test_transformed_rfe, feature_names=selected_features)

# Model predictions and evaluation
y_pred = model.predict(X_test_transformed_rfe)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'\nModel Performance')
print(f"Test MSE: {mse}")
print(f"Test RMSE: {rmse}")
print(f"Test MAE: {mae}")
print(f"Test R² Score: {r2}")

# Plot residuals
residuals = y_test - y_pred
plt.scatter(y_pred, residuals)
plt.hlines(y=0, xmin=y_pred.min(), xmax=y_pred.max(), colors='red')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residuals vs Predicted Values')
plt.show()



ValueError: Found input variables with inconsistent numbers of samples: [17471, 322]

In [32]:
# Load the hyperparameters from the JSON file
with open('/Users/skylerwilson/Desktop/PartsWise/co-pilot-v1/Dashboard/Models/demand_predictor/general_model_details.json', 'r') as file:
    hyperparameters = json.load(file)

# Print the hyperparameters
print("Hyperparameters saved in the JSON file:")
print(hyperparameters)

Hyperparameters saved in the JSON file:
{'selected_features': ['normalized_seasonal_score', 'quantity', 'months_no_sale', 'margin', 'special_orders_ytd', 'cost_per_unit', 'gross_profit', 'roi', 'rolling_3m_sales', '1m_days_supply', 'turnover', 'sell_through_rate', 'days_of_inventory_outstanding'], 'best_hyperparameters': {'colsample_bytree': 0.5430197098620704, 'gamma': 0.2709124173353241, 'huber_slope': 0.2601606035840498, 'learning_rate': 0.048837795279661096, 'max_delta_step': 10, 'max_depth': 7, 'min_child_weight': 3, 'n_estimators': 750, 'reg_alpha': 0.2166228090437864, 'reg_lambda': 1.015388986935639, 'subsample': 0.9272755187944987}}


In [28]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import numpy as np

# Load the data
df = pd.read_csv("/Users/skylerwilson/Desktop/PartsWise/co-pilot-v1/data/processed_data/parts_data_prepared.csv")

# Select the features (X) and target (y) for modeling
feature_cols = [col for col in df.columns if col not in {'part_number', 'description', 'supplier_name',
                                                         'sales_last_jan','sales_last_feb', 'sales_last_mar', 'sales_last_apr', 'sales_last_may',
                                                         'sales_last_jun', 'sales_last_jul', 'sales_last_aug', 'sales_last_sep',
                                                         'sales_last_oct', 'sales_last_nov', 'sales_last_dec', 'sales_jan',
                                                         'sales_feb', 'sales_mar', 'sales_apr', 'sales_may', 'sales_jun', 
                                                         'sales_jul', 'sales_aug', 'sales_sep', 'sales_oct', 'sales_nov', 
                                                         'sales_dec', 'sales_revenue', 'cogs', 'price', 'rolling_12m_sales'}]
X = df[feature_cols]

# Add a constant to the DataFrame
df = sm.add_constant(X)

# Calculate VIF
vif_data = pd.DataFrame()
vif_data["feature"] = df.columns
vif_data["VIF"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]

# Display the VIF for each feature
print(vif_data)


                          feature        VIF
0                           const  12.488638
1       normalized_seasonal_score   2.540796
2                        quantity   5.002715
3                  months_no_sale   1.079207
4            quantity_ordered_ytd   1.064776
5                negative_on_hand   1.163495
6                          margin   3.376943
7              special_orders_ytd   1.041023
8                   cost_per_unit   3.106045
9                      total_cost   1.280671
10              margin_percentage   1.177468
11                   gross_profit   1.734802
12                            roi   1.043944
13               rolling_3m_sales   3.511741
14                12m_days_supply   4.011494
15                 3m_days_supply   4.446802
16                 1m_days_supply   4.611917
17                       turnover   1.017243
18                    3m_turnover   1.044536
19              sell_through_rate   1.128851
20  days_of_inventory_outstanding   1.441075
21        